In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from models.cycle_gan_model_parallel import CycleGAN
from data import create_dataset, find_dataset_using_name
import argparse
import torch

In [19]:
device = torch.device(0)
device

device(type='cuda', index=0)

In [20]:
dataset_class = find_dataset_using_name('unaligned')
opt = argparse.Namespace()
opt.dataroot = '/fs/nexus-scratch/vla/datasets/maps'
opt.max_dataset_size = float('inf')
opt.input_nc = 3
opt.output_nc = 3
opt.direction = 'AtoB'
opt.phase = 'train'
opt.preprocess = 'resize_and_crop'
opt.load_size = 286
opt.crop_size = 256
opt.no_flip = False
opt.serial_batches = False

opt

Namespace(dataroot='/fs/nexus-scratch/vla/datasets/maps', max_dataset_size=inf, input_nc=3, output_nc=3, direction='AtoB', phase='train', preprocess='resize_and_crop', load_size=286, crop_size=256, no_flip=False, serial_batches=False)

In [21]:
dataset = dataset_class(opt)

In [22]:
len(dataset)

1096

In [23]:
dataset[0]

{'A': tensor([[[-0.1451, -0.2314, -0.2157,  ..., -0.7490, -0.6863, -0.8667],
          [-0.1765, -0.2314, -0.2392,  ..., -0.3176, -0.3961, -0.7490],
          [-0.2078, -0.2235, -0.2314,  ..., -0.2392, -0.3647, -0.4431],
          ...,
          [-0.7412, -0.7569, -0.7569,  ..., -0.1843, -0.0510, -0.0431],
          [-0.7725, -0.7569, -0.7647,  ..., -0.3725, -0.2863, -0.1216],
          [-0.7176, -0.7490, -0.7804,  ..., -0.3255, -0.3569, -0.2392]],
 
         [[-0.1529, -0.2392, -0.2235,  ..., -0.7255, -0.6627, -0.8510],
          [-0.1843, -0.2392, -0.2471,  ..., -0.3176, -0.3961, -0.7569],
          [-0.2157, -0.2314, -0.2392,  ..., -0.2706, -0.3961, -0.4667],
          ...,
          [-0.7490, -0.7490, -0.7490,  ..., -0.2000, -0.0667, -0.0588],
          [-0.7725, -0.7490, -0.7569,  ..., -0.4118, -0.3098, -0.1451],
          [-0.7255, -0.7412, -0.7725,  ..., -0.3725, -0.3961, -0.2706]],
 
         [[-0.1922, -0.2784, -0.2627,  ..., -0.7804, -0.7255, -0.9137],
          [-0.2235, -0.

In [24]:
model = CycleGAN(device=device)

initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal


In [25]:
# NOTE: Defined as G in CycleGAN paper
model.netG_A

ResnetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
     

In [26]:
# NOTE: Defined as F in CycleGAN paper
model.netG_B

ResnetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
     

In [27]:
# NOTE: define as D_Y in CycleGAN paper
model.netD_A

NLayerDiscriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [28]:
# NOTE: define as D_X in CycleGAN paper
model.netD_B

NLayerDiscriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [29]:
# start with just using a basic cuda stream --> then expand to more sophisticated distribution

In [30]:
schedule = torch.profiler.schedule(wait=5, warmup=5, active=20, repeat=3)
trace_handler = torch.profiler.tensorboard_trace_handler(dir_name = './dist_traces', use_gzip=False)

In [ ]:
with torch.profiler.profile(
            activities=[
                torch.profiler.ProfilerActivity.CPU,
                torch.profiler.ProfilerActivity.CUDA,
            ],
            schedule=schedule,
            record_shapes=True,
            profile_memory=True,
            with_stack=True,          # NOTE: set to True for memory profiling, TODO: currently setting to True causes a runtime error: https://github.com/pytorch/pytorch/pull/150102
            on_trace_ready=trace_handler,
            # experimental_config = experimental_config
) as prof:

    # one epoch for now
    for i in range(1):

        # TODO: adapt update learning rates? maybe unecessary
        for i, data in enumerate(dataset):

            if i % 100 == 0:
                break
            # forward pass
            model(data)
            model.optimize_parameters()
            prof.step()
        
        break

    # Construct the memory timeline HTML plot.
    # prof.export_memory_timeline(f"dist_timeline.html", device="cuda:0")

    # metrics table 
    print(prof.key_averages().table(sort_by="self_cuda_memory_usage"))


AssertionError: 